In [41]:
%load_ext autoreload
%autoreload 2
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [42]:
import sys
import glob
import pandas as pd
import os
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import open_spiel.python.examples.ubc_dispatch as dispatch
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from sklearn.model_selection import ParameterGrid
from pathlib import Path
import torch.nn.functional as F
import torch

In [43]:
game_dir = os.environ["CLOCK_AUCTION_CONFIG_DIR"]

In [44]:
PREFIX = 'jun24'
sats_games = [f'{PREFIX}/' + os.path.basename(x).replace('.json','') for x in glob.glob(f'{game_dir}/{PREFIX}/*')]
sats_games

['jun24/jun24_0_base',
 'jun24/jun24_3_medium_speed',
 'jun24/jun24_1_undersell_allowed',
 'jun24/jun24_1_medium_speed',
 'jun24/jun24_1_base',
 'jun24/jun24_4_base',
 'jun24/jun24_3_undersell_allowed',
 'jun24/jun24_1_no_activity',
 'jun24/jun24_2_high_speed',
 'jun24/jun24_4_no_activity',
 'jun24/jun24_4_medium_speed',
 'jun24/jun24_0_medium_speed',
 'jun24/jun24_3_base',
 'jun24/jun24_2_base',
 'jun24/jun24_3_no_activity',
 'jun24/jun24_2_medium_speed',
 'jun24/jun24_0_no_activity',
 'jun24/jun24_0_high_speed',
 'jun24/jun24_1_high_speed',
 'jun24/jun24_4_high_speed',
 'jun24/jun24_4_undersell_allowed',
 'jun24/jun24_2_undersell_allowed',
 'jun24/jun24_2_no_activity',
 'jun24/jun24_3_high_speed',
 'jun24/jun24_0_undersell_allowed']

# For running the real dumb test case

In [45]:
pgrid = ParameterGrid([
    {
        # Defaults
        'game': [g for g in sats_games]# if 'hide' not in g and 'imperfect' not in g]
    },
])
print(len(pgrid))

25


In [12]:
num_iters = 10_000_000
time_limit_seconds = 3600 * 4
for config in pgrid:
    CONFIG = 'cfr_external' # The YML network config folder for the main jobs
    N_SEEDS = 3 # Number of seeds of each experiment to launch
    extra_name = ''
    overrides = f'--overwrite_db True --eval_every 250_000 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 5000 --time_limit_seconds {time_limit_seconds}'      
    dispatch.dispatch_experiments(CONFIG, base_job_name='jun23external', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='cfr')

Submitted batch job 86951
Submitted batch job 86952
Submitted batch job 86953
Submitted batch job 86954
Submitted batch job 86955
Submitted batch job 86956
Submitted batch job 86957
Submitted batch job 86958
Submitted batch job 86959
Submitted batch job 86960
Submitted batch job 86961
Submitted batch job 86962
Submitted batch job 86963
Submitted batch job 86964
Submitted batch job 86965
Submitted batch job 86966
Submitted batch job 86967
Submitted batch job 86968
Submitted batch job 86969
Submitted batch job 86970
Submitted batch job 86971
Submitted batch job 86972
Submitted batch job 86973
Submitted batch job 86974
Submitted batch job 86975


In [46]:
num_iters = 30_000_000
time_limit_seconds = 3600 * 4
for config in pgrid:
    CONFIG = 'cfr_outcome' # The YML network config folder for the main jobs
    N_SEEDS = 3 # Number of seeds of each experiment to launch
    extra_name = ''
    overrides = f'--overwrite_db True --eval_every 5_000_000 --eval_zero false --total_timesteps {num_iters} --dispatch_br false --report_freq 5000'      
    dispatch.dispatch_experiments(CONFIG, base_job_name='jun23outcome', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name, alg='cfr')

Submitted batch job 86976
Submitted batch job 86977
Submitted batch job 86978
Submitted batch job 86979
Submitted batch job 86980
Submitted batch job 86981
Submitted batch job 86982
Submitted batch job 86983
Submitted batch job 86984
Submitted batch job 86985
Submitted batch job 86986
Submitted batch job 86987
Submitted batch job 86988
Submitted batch job 86989
Submitted batch job 86990
Submitted batch job 86991
Submitted batch job 86992
Submitted batch job 86993
Submitted batch job 86994
Submitted batch job 86995
Submitted batch job 86996
Submitted batch job 86997
Submitted batch job 86998
Submitted batch job 86999
Submitted batch job 87000
Submitted batch job 87001
Submitted batch job 87002
Submitted batch job 87003
Submitted batch job 87004
Submitted batch job 87005
Submitted batch job 87006
Submitted batch job 87007
Submitted batch job 87008
Submitted batch job 87009
Submitted batch job 87010
Submitted batch job 87011
Submitted batch job 87012
Submitted batch job 87013
Submitted ba

In [ ]:
# for config in pgrid:
#     CONFIG = 'ppo_no_ent_jun14'
#     # CONFIG = 'ppo_may31_23' # The YML network config folder for the main jobs
#     # BR_CONFIG = 'ppo_may31_23' 
#     N_SEEDS = 3 # Number of seeds of each experiment to launch
#     extra_name = ''
#     overrides = f'--overwrite_db True --br_portfolio_path {BR_CONFIG} --eval_every 999_000_000 --eval_zero false --total_timesteps 5_000_000 --dispatch_br false --br_overrides "--total_timesteps 1_000_000"'        
#     dispatch.dispatch_experiments(CONFIG, base_job_name='jun14ent_3', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name)

In [40]:
!squeue --format="%.18i %.90j %.2t .%.10M %.6D %R"

             JOBID                                                                                       NAME ST .      TIME  NODES NODELIST(REASON)


In [ ]:
def check_queue():
    !ssh borg.cs.ubc.ca '/opt/slurm/bin/squeue -u newmanne --format="%.18i %.90j %.2t .%.10M %.6D %R"'


In [4]:
!squeue --format "%.18i %.90j %.2t .%.10M %.6D %R"

             JOBID                                                                                       NAME ST .      TIME  NODES NODELIST(REASON)


In [ ]:
check_queue()

In [37]:
!squeue | wc -l

101


In [ ]:
!squeue | grep "may30" | wc -l

In [39]:
!scancel -u ubuntu

In [ ]:
# !scancel  46891 46893

In [ ]:
!rm -rf /shared/outputs/minitest/
try:
    Experiment.objects.get(name='minitest').delete()
except:
    pass
dispatch.dispatch_experiments('feb2', base_job_name='minitest', game_name='tiny', overrides = '--br_portfolio_path feb2 --eval_every 500 --num_training_episodes 1_000 --br_overrides "--num_training_episodes 1_000" --eval_overrides "--num_samples 100"')

In [ ]:
BestResponse.objects.last()

In [ ]:
Experiment.objects.all()

In [ ]:
!sacct --starttime 2022-03-11 --format=User,JobID,Jobname,partition,state,time,start,end,elapsed,MaxRss,MaxVMSize,nnodes,ncpus,nodelist

In [ ]:
Evaluation.objects.last()

In [ ]:
os.system("ssh borg.cs.ubc.ca '/opt/slurm/bin/sbatch'")